# Full Dataset: 

The first 50 are the primary variables including all of the dummy variables. The next 560 are interactions with the dummy variables. The final variables are the adjacency variables that aren't in the primary variable section. 

In [1]:
import numpy as np 
import pandas as pd
import re
from IPython.display import display, HTML

In [2]:
from IPython.display import display, Markdown
import bokeh

from bokeh.io import show, output_notebook, output_file
from bokeh.layouts import gridplot
from bokeh.charts import Bar
from bokeh.charts.attributes import cat
from bokeh.models import HoverTool
output_notebook ()

Loading BokehJS ...

In [3]:
combined = pd.read_csv("new data/Final_Project_Variables - combined.csv")
print(combined.shape)

(124011, 825)


In [72]:
train_all = pd.read_csv("new data/Final_Project_Variables - train.csv")
locations = pd.read_csv("new data/Google Locations - train.csv")

In [73]:
train_all = train_all.merge(locations[["observation","location_code"]], on="observation")
train_all.head()

,observation,listing_id,interest_code,days_between_adj_5,log_price_adj_3,interest_adj_13,feature_count_adj_25,bathrooms_adj_4,descript_len_adj_11,bedrooms_adj_3,...,bathrooms_adj_22,bathrooms_adj_23,bathrooms_adj_24,bathrooms_adj_25,bathrooms_adj_26,bathrooms_adj_27,bathrooms_adj_28,bathrooms_adj_29,bathrooms_adj_30,location_code
0,4,7170325,2,1318,23.485804,19,130,4.0,5398,4,...,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,8
1,6,7092344,1,1173,24.941265,18,168,6.0,7174,7,...,24.0,26.0,27.0,28.0,29.0,30.0,31.0,33.0,34.0,2
2,9,7158677,2,1170,24.302001,21,127,4.0,6583,5,...,26.0,27.0,28.0,31.0,34.0,35.0,36.0,37.0,38.0,15
3,10,7211212,2,1189,23.654460,15,46,4.5,5388,5,...,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,8
4,15,7225292,1,1207,25.326354,17,195,6.0,6230,5,...,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,2


In [74]:
del train_all["observation.1"]
del train_all["listing_id.1"]

In [75]:
print(train_all.shape)

(49352, 824)


In [76]:
holdout = pd.read_csv("new data/Final_Project_Variables - test.csv")
locations_test = pd.read_csv("new data/Google Locations - test.csv")
holdout = holdout.merge(locations_test[["observation","location_code"]], on="observation")
del holdout["observation.1"]
del holdout["listing_id.1"]
print(holdout.shape)

(74659, 824)


#### Build dataset with reduced adjacencies and no interaction terms: 

In [9]:
train_all_small = pd.read_csv("new data/no_corr_train.csv")

In [10]:
train_all_small = train_all_small .merge(locations[["observation","location_code"]], on="observation")

In [11]:
#delete random extra column: 
del train_all_small["Unnamed: 0"]
train_all_small.head()

,observation,listing_id,interest_code,mgmt_list_count,feature_count,descript_len,photo_count,bedrooms,bathrooms,log_price,...,photo_count_adj_30,log_price_adj_3,log_price_adj_12,bedrooms_adj_1,bedrooms_adj_3,bathrooms_adj_1,bathrooms_adj_4,bathrooms_adj_7,bathrooms_adj_25,location_code
0,4,7170325,2,294,7,530,12,1,1.0,7.783224,...,235,23.485804,93.244562,1,4,1.0,4.0,7.0,25.0,8
1,6,7092344,1,64,6,804,6,2,1.0,8.242756,...,131,24.941265,98.428382,2,7,1.0,6.0,9.0,28.0,2
2,9,7158677,2,265,6,776,6,2,1.0,8.159089,...,136,24.302001,98.417272,2,5,1.0,4.0,7.0,31.0,15
3,10,7211212,2,235,0,565,5,3,1.5,8.006368,...,148,23.654460,93.360586,1,5,1.0,4.5,7.5,27.0,8
4,15,7225292,1,135,4,321,4,0,1.0,7.935587,...,153,25.326354,98.639056,2,5,2.0,6.0,10.0,28.0,2


In [12]:
holdout_small = pd.read_csv("new data/no_corr_test.csv")
holdout_small = holdout_small.merge(locations_test[["observation","location_code"]], on="observation")
#delete random extra column: 
del holdout_small["Unnamed: 0"]

*Outliers* (all numbers expressed as indices in the training set, not as observation ids)

- Feature_count         27648 28418
- Description length    48522
- Bedrooms              4440 10673 11387 41667    (have 7 or 8 bedrooms)
- Bathrooms             41576 (has 10 bathrooms)
- log price             12948
- Distance adj1         6479 29013 31264 41723      6479(definitely remove!)
- Desciption len2       13436 17719

In [79]:
#remove outliers:
outliers = [  27648, 28418,48522, 4440 ,10673, 11387 ,41667, 41576 ,12948,6479, 29013, 31264, 41723, 6479,13436, 17719 ]
outliers = [x-1 for x in outliers]

train_all_small = train_all_small.drop(outliers)
train_all = train_all.drop(outliers)

In [14]:
#list(train_all.columns)

### All training samples Class Distribution: 

In [80]:
interest = train_all["interest_code"]
print("percentage of Lows: ", len(interest[interest== 1])/len(interest) * 100)
print("percentage of Mediums: ", len(interest[interest== 2])/len(interest) * 100)
print("percentage of Highs: ", len(interest[interest== 3])/len(interest) * 100)

percentage of Lows:  69.4711879522468
percentage of Mediums:  22.747633621825404
percentage of Highs:  7.781178425927802


### Data prep:

First, recode binary variables as -1 vs 1 instead of as 0 vs 1 to center the predictor around 0. 

"The difference is due to simple geometry. The initial hyperplanes pass fairly near the origin. If the data are centered near the origin (as with {-1,1} coding), the initial hyperplanes will cut through the data in a variety of directions. If the data are offset from the origin (as with {0,1} coding), many of the initial hyperplanes will miss the data entirely, and those that pass through the data will provide a only a limited range of directions, making it difficult to find local optima that use hyperplanes that go in different directions."

In [16]:
train_all_small.loc[train_all_small["one"]==0, "one"] = -1
train_all_small.loc[train_all_small["two"]==0, "two"] = -1
train_all_small.loc[train_all_small["three"]==0, "three"] = -1
train_all_small.loc[train_all_small["five"]==0, "five"] = -1
train_all_small.loc[train_all_small["six"]==0, "six"] = -1
train_all_small.loc[train_all_small["seven"]==0, "seven"] = -1
train_all_small.loc[train_all_small["Doorman"]==0, "Doorman"] = -1
train_all_small.loc[train_all_small[ "Dogs_Allowed"]==0,  "Dogs_Allowed"] = -1
train_all_small.loc[train_all_small["Cats_Allowed"]==0, "Cats_Allowed"] = -1
train_all_small.loc[train_all_small["Elevator"  ]==0, "Elevator"  ] = -1
train_all_small.loc[train_all_small["Balcony" ]==0,"Balcony" ] = -1
train_all_small.loc[train_all_small["Outdoors"]==0, "Outdoors"] = -1
train_all_small.loc[train_all_small["Hardwood" ]==0, "Hardwood" ] = -1
train_all_small.loc[train_all_small["Internet_Mention" ]==0, "Internet_Mention" ] = -1
train_all_small.loc[train_all_small["Loft" ]==0,"Loft"  ] = -1
train_all_small.loc[train_all_small["New" ]==0,"New"  ] = -1
train_all_small.loc[train_all_small["No_fee"  ]==0, "No_fee"  ] = -1
train_all_small.loc[train_all_small["Gym"  ]==0,"Gym"   ] = -1
train_all_small.loc[train_all_small["Parking"]==0, "Parking"] = -1
train_all_small.loc[train_all_small["Storage"]==0, "Storage"  ] = -1
train_all_small.loc[train_all_small["Sunlight" ]==0,"Sunlight"  ] = -1
train_all_small.loc[train_all_small["Pool"   ]==0, "Pool"   ] = -1
train_all_small.loc[train_all_small[ "Laundry_Room"  ]==0, "Laundry_Room"   ] = -1
train_all_small.loc[train_all_small["Wash_Dry"   ]==0, "Wash_Dry"   ] = -1
train_all_small.loc[train_all_small["Dishwasher" ]==0,"Dishwasher"  ] = -1
train_all_small.loc[train_all_small[ "Prewar" ]==0,  "Prewar" ] = -1
train_all_small.loc[train_all_small["AirCon"]==0, "AirCon"   ] = -1


holdout_small.loc[holdout_small["one"]==0, "one"] = -1
holdout_small.loc[holdout_small["two"]==0, "two"] = -1
holdout_small.loc[holdout_small["three"]==0, "three"] = -1
holdout_small.loc[holdout_small["five"]==0, "five"] = -1
holdout_small.loc[holdout_small["six"]==0, "six"] = -1
holdout_small.loc[holdout_small["seven"]==0, "seven"] = -1
holdout_small.loc[holdout_small["Doorman"]==0, "Doorman"] = -1
holdout_small.loc[holdout_small[ "Dogs_Allowed"]==0,  "Dogs_Allowed"] = -1
holdout_small.loc[holdout_small["Cats_Allowed"]==0, "Cats_Allowed"] = -1
holdout_small.loc[holdout_small["Elevator"  ]==0, "Elevator"  ] = -1
holdout_small.loc[holdout_small["Balcony" ]==0,"Balcony" ] = -1
holdout_small.loc[holdout_small["Outdoors"]==0, "Outdoors"] = -1
holdout_small.loc[holdout_small["Hardwood" ]==0, "Hardwood" ] = -1
holdout_small.loc[holdout_small["Internet_Mention" ]==0, "Internet_Mention" ] = -1
holdout_small.loc[holdout_small["Loft" ]==0,"Loft"  ] = -1
holdout_small.loc[holdout_small["New" ]==0,"New"  ] = -1
holdout_small.loc[holdout_small["No_fee"  ]==0, "No_fee"  ] = -1
holdout_small.loc[holdout_small["Gym"  ]==0,"Gym"   ] = -1
holdout_small.loc[holdout_small["Parking"]==0, "Parking"] = -1
holdout_small.loc[holdout_small["Storage"]==0, "Storage"  ] = -1
holdout_small.loc[holdout_small["Sunlight" ]==0,"Sunlight"  ] = -1
holdout_small.loc[holdout_small["Pool"   ]==0, "Pool"   ] = -1
holdout_small.loc[holdout_small[ "Laundry_Room"  ]==0, "Laundry_Room"   ] = -1
holdout_small.loc[holdout_small["Wash_Dry"   ]==0, "Wash_Dry"   ] = -1
holdout_small.loc[holdout_small["Dishwasher" ]==0,"Dishwasher"  ] = -1
holdout_small.loc[holdout_small[ "Prewar" ]==0,  "Prewar" ] = -1
holdout_small.loc[holdout_small["AirCon"]==0, "AirCon"   ] = -1

In [81]:
#do the -1/1 encoding for the full dataset: 

train_all.loc[train_all["one"]==0, "one"] = -1
train_all.loc[train_all["two"]==0, "two"] = -1
train_all.loc[train_all["three"]==0, "three"] = -1
train_all.loc[train_all["five"]==0, "five"] = -1
train_all.loc[train_all["six"]==0, "six"] = -1
train_all.loc[train_all["seven"]==0, "seven"] = -1
train_all.loc[train_all["Doorman"]==0, "Doorman"] = -1
train_all.loc[train_all[ "Dogs_Allowed"]==0,  "Dogs_Allowed"] = -1
train_all.loc[train_all["Cats_Allowed"]==0, "Cats_Allowed"] = -1
train_all.loc[train_all["Elevator"  ]==0, "Elevator"  ] = -1
train_all.loc[train_all["Balcony" ]==0,"Balcony" ] = -1
train_all.loc[train_all["Outdoors"]==0, "Outdoors"] = -1
train_all.loc[train_all["Hardwood" ]==0, "Hardwood" ] = -1
train_all.loc[train_all["Internet_Mention" ]==0, "Internet_Mention" ] = -1
train_all.loc[train_all["Loft" ]==0,"Loft"  ] = -1
train_all.loc[train_all["New" ]==0,"New"  ] = -1
train_all.loc[train_all["No_fee"  ]==0, "No_fee"  ] = -1
train_all.loc[train_all["Gym"  ]==0,"Gym"   ] = -1
train_all.loc[train_all["Parking"]==0, "Parking"] = -1
train_all.loc[train_all["Storage"]==0, "Storage"  ] = -1
train_all.loc[train_all["Sunlight" ]==0,"Sunlight"  ] = -1
train_all.loc[train_all["Pool"   ]==0, "Pool"   ] = -1
train_all.loc[train_all[ "Laundry_Room"  ]==0, "Laundry_Room"   ] = -1
train_all.loc[train_all["Wash_Dry"   ]==0, "Wash_Dry"   ] = -1
train_all.loc[train_all["Dishwasher" ]==0,"Dishwasher"  ] = -1
train_all.loc[train_all[ "Prewar" ]==0,  "Prewar" ] = -1
train_all.loc[train_all["AirCon"]==0, "AirCon"   ] = -1


holdout.loc[holdout["one"]==0, "one"] = -1
holdout.loc[holdout["two"]==0, "two"] = -1
holdout.loc[holdout["three"]==0, "three"] = -1
holdout.loc[holdout["five"]==0, "five"] = -1
holdout.loc[holdout["six"]==0, "six"] = -1
holdout.loc[holdout["seven"]==0, "seven"] = -1
holdout.loc[holdout["Doorman"]==0, "Doorman"] = -1
holdout.loc[holdout[ "Dogs_Allowed"]==0,  "Dogs_Allowed"] = -1
holdout.loc[holdout["Cats_Allowed"]==0, "Cats_Allowed"] = -1
holdout.loc[holdout["Elevator"  ]==0, "Elevator"  ] = -1
holdout.loc[holdout["Balcony" ]==0,"Balcony" ] = -1
holdout.loc[holdout["Outdoors"]==0, "Outdoors"] = -1
holdout.loc[holdout["Hardwood" ]==0, "Hardwood" ] = -1
holdout.loc[holdout["Internet_Mention" ]==0, "Internet_Mention" ] = -1
holdout.loc[holdout["Loft" ]==0,"Loft"  ] = -1
holdout.loc[holdout["New" ]==0,"New"  ] = -1
holdout.loc[holdout["No_fee"  ]==0, "No_fee"  ] = -1
holdout.loc[holdout["Gym"  ]==0,"Gym"   ] = -1
holdout.loc[holdout["Parking"]==0, "Parking"] = -1
holdout.loc[holdout["Storage"]==0, "Storage"  ] = -1
holdout.loc[holdout["Sunlight" ]==0,"Sunlight"  ] = -1
holdout.loc[holdout["Pool"   ]==0, "Pool"   ] = -1
holdout.loc[holdout[ "Laundry_Room"  ]==0, "Laundry_Room"   ] = -1
holdout.loc[holdout["Wash_Dry"   ]==0, "Wash_Dry"   ] = -1
holdout.loc[holdout["Dishwasher" ]==0,"Dishwasher"  ] = -1
holdout.loc[holdout[ "Prewar" ]==0,  "Prewar" ] = -1
holdout.loc[holdout["AirCon"]==0, "AirCon"   ] = -1

In [17]:
#Because MLP is sensitive to scaling, scale all the continuous predictors 

from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin

#This custom function will scale only the continuous predictors:
class CustomScaler(BaseEstimator,TransformerMixin): 
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        self.scaler = StandardScaler(copy,with_mean,with_std)
        self.columns = columns

    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        return self

    def transform(self, X, y=None, copy=None):
        init_col_order = X.columns
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns, index=X.index)
        X_not_scaled = X.ix[:,~X.columns.isin(self.columns)]
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]


In [67]:
#scale all the predictors in the train and holdout data:

continuous_cols =  ['mgmt_list_count','feature_count','descript_len','photo_count','bedrooms','bathrooms','log_price',
 'log_days_between','website','interest_adj_13','interest_adj_1', 'interest_adj_2','interest_adj_3',
 'distance_adj_1','distance_adj_2','distance_adj_30','days_between_adj_2','days_between_adj_3',
 'days_between_adj_30','feature_count_adj_25','descript_len_adj_1','descript_len_adj_2','descript_len_adj_11',
 'descript_len_adj_27','photo_count_adj_1','photo_count_adj_30','log_price_adj_3','log_price_adj_12',
 'bedrooms_adj_1','bedrooms_adj_3','bathrooms_adj_1','bathrooms_adj_4','bathrooms_adj_7','bathrooms_adj_25']

scale = CustomScaler(columns=continuous_cols )

train_all_scale_small = scale.fit_transform(train_all_small)
holdout_scale_small = scale.fit_transform(holdout_small)  

In [94]:
continuous_cols2 = ['days_between_adj_5','log_price_adj_3','interest_adj_13','feature_count_adj_25','bathrooms_adj_4',
 'descript_len_adj_11','bedrooms_adj_3','mgmt_list_count','feature_count','descript_len','photo_count','bedrooms','bathrooms','log_price','log_price_sq',
 'log_price__interest_adj_13', 'log_days_between','website','days_between_adj_5__website',
 'days_between_adj_5__one',  'days_between_adj_5__two', 'days_between_adj_5__three',
 'days_between_adj_5__four',
 'days_between_adj_5__five',
 'days_between_adj_5__six',
 'days_between_adj_5__seven',
 'days_between_adj_5__Doorman',
 'days_between_adj_5__Dogs_Allowed',
 'days_between_adj_5__Cats_Allowed',
 'days_between_adj_5__Pets_Allowed',
 'days_between_adj_5__Elevator',
 'days_between_adj_5__Balcony',
 'days_between_adj_5__Outdoors',
 'days_between_adj_5__Hardwood',
 'days_between_adj_5__Internet_Mention',
 'days_between_adj_5__Loft',
 'days_between_adj_5__New',
 'days_between_adj_5__No_fee',
 'days_between_adj_5__Gym',
 'days_between_adj_5__Parking',
 'days_between_adj_5__Storage',
 'days_between_adj_5__Sunlight',
 'days_between_adj_5__Pool',
 'days_between_adj_5__Laundry_Room',
 'days_between_adj_5__Wash_Dry',
 'days_between_adj_5__Dishwasher',
 'days_between_adj_5__Prewar',
 'days_between_adj_5__AirCon',
 'log_price_adj_3__website',
 'log_price_adj_3__one',
 'log_price_adj_3__two',
 'log_price_adj_3__three',
 'log_price_adj_3__four',
 'log_price_adj_3__five',
 'log_price_adj_3__six', 'log_price_adj_3__seven', 'log_price_adj_3__Doorman', 'log_price_adj_3__Dogs_Allowed', 'log_price_adj_3__Cats_Allowed',
 'log_price_adj_3__Pets_Allowed', 'log_price_adj_3__Elevator',
 'log_price_adj_3__Balcony',
 'log_price_adj_3__Outdoors',
 'log_price_adj_3__Hardwood',
 'log_price_adj_3__Internet_Mention',
 'log_price_adj_3__Loft',
 'log_price_adj_3__New',
 'log_price_adj_3__No_fee',
 'log_price_adj_3__Gym',
 'log_price_adj_3__Parking',
 'log_price_adj_3__Storage',
 'log_price_adj_3__Sunlight',
 'log_price_adj_3__Pool',
 'log_price_adj_3__Laundry_Room',
 'log_price_adj_3__Wash_Dry',
 'log_price_adj_3__Dishwasher',
 'log_price_adj_3__Prewar',
 'log_price_adj_3__AirCon',
 'interest_adj_13__website',
 'interest_adj_13__one',
 'interest_adj_13__two',
 'interest_adj_13__three',
 'interest_adj_13__four',
 'interest_adj_13__five',
 'interest_adj_13__six',
 'interest_adj_13__seven',
 'interest_adj_13__Doorman',
 'interest_adj_13__Dogs_Allowed',
 'interest_adj_13__Cats_Allowed',
 'interest_adj_13__Pets_Allowed',
 'interest_adj_13__Elevator',
 'interest_adj_13__Balcony',
 'interest_adj_13__Outdoors',
 'interest_adj_13__Hardwood',
 'interest_adj_13__Internet_Mention',
 'interest_adj_13__Loft',
 'interest_adj_13__New',
 'interest_adj_13__No_fee',
 'interest_adj_13__Gym',
 'interest_adj_13__Parking',
 'interest_adj_13__Storage',
 'interest_adj_13__Sunlight',
 'interest_adj_13__Pool',
 'interest_adj_13__Laundry_Room',
 'interest_adj_13__Wash_Dry',
 'interest_adj_13__Dishwasher',
 'interest_adj_13__Prewar',
 'interest_adj_13__AirCon',
 'feature_count_adj_25__website',
 'feature_count_adj_25__one',
 'feature_count_adj_25__two',
 'feature_count_adj_25__three',
 'feature_count_adj_25__four',
 'feature_count_adj_25__five',
 'feature_count_adj_25__six',
 'feature_count_adj_25__seven',
 'feature_count_adj_25__Doorman',
 'feature_count_adj_25__Dogs_Allowed',
 'feature_count_adj_25__Cats_Allowed',
 'feature_count_adj_25__Pets_Allowed',
 'feature_count_adj_25__Elevator',
 'feature_count_adj_25__Balcony',
 'feature_count_adj_25__Outdoors',
 'feature_count_adj_25__Hardwood',
 'feature_count_adj_25__Internet_Mention',
 'feature_count_adj_25__Loft',
 'feature_count_adj_25__New',
 'feature_count_adj_25__No_fee',
 'feature_count_adj_25__Gym',
 'feature_count_adj_25__Parking',
 'feature_count_adj_25__Storage',
 'feature_count_adj_25__Sunlight',
 'feature_count_adj_25__Pool',
 'feature_count_adj_25__Laundry_Room',
 'feature_count_adj_25__Wash_Dry',
 'feature_count_adj_25__Dishwasher',
 'feature_count_adj_25__Prewar',
 'feature_count_adj_25__AirCon',
 'bathrooms_adj_4__website',
 'bathrooms_adj_4__one',
 'bathrooms_adj_4__two',
 'bathrooms_adj_4__three',
 'bathrooms_adj_4__four',
 'bathrooms_adj_4__five',
 'bathrooms_adj_4__six',
 'bathrooms_adj_4__seven',
 'bathrooms_adj_4__Doorman',
 'bathrooms_adj_4__Dogs_Allowed',
 'bathrooms_adj_4__Cats_Allowed',
 'bathrooms_adj_4__Pets_Allowed',
 'bathrooms_adj_4__Elevator',
 'bathrooms_adj_4__Balcony',
 'bathrooms_adj_4__Outdoors',
 'bathrooms_adj_4__Hardwood',
 'bathrooms_adj_4__Internet_Mention',
 'bathrooms_adj_4__Loft',
 'bathrooms_adj_4__New',
 'bathrooms_adj_4__No_fee',
 'bathrooms_adj_4__Gym',
 'bathrooms_adj_4__Parking',
 'bathrooms_adj_4__Storage',
 'bathrooms_adj_4__Sunlight',
 'bathrooms_adj_4__Pool',
 'bathrooms_adj_4__Laundry_Room',
 'bathrooms_adj_4__Wash_Dry',
 'bathrooms_adj_4__Dishwasher',
 'bathrooms_adj_4__Prewar',
 'bathrooms_adj_4__AirCon',
 'descript_len_adj_11__website',
 'descript_len_adj_11__one',
 'descript_len_adj_11__two',
 'descript_len_adj_11__three',
 'descript_len_adj_11__four',
 'descript_len_adj_11__five',
 'descript_len_adj_11__six',
 'descript_len_adj_11__seven',
 'descript_len_adj_11__Doorman',
 'descript_len_adj_11__Dogs_Allowed',
 'descript_len_adj_11__Cats_Allowed',
 'descript_len_adj_11__Pets_Allowed',
 'descript_len_adj_11__Elevator',
 'descript_len_adj_11__Balcony',
 'descript_len_adj_11__Outdoors',
 'descript_len_adj_11__Hardwood',
 'descript_len_adj_11__Internet_Mention',
 'descript_len_adj_11__Loft',
 'descript_len_adj_11__New',
 'descript_len_adj_11__No_fee',
 'descript_len_adj_11__Gym',
 'descript_len_adj_11__Parking',
 'descript_len_adj_11__Storage',
 'descript_len_adj_11__Sunlight',
 'descript_len_adj_11__Pool',
 'descript_len_adj_11__Laundry_Room',
 'descript_len_adj_11__Wash_Dry',
 'descript_len_adj_11__Dishwasher',
 'descript_len_adj_11__Prewar',
 'descript_len_adj_11__AirCon',
 'bedrooms_adj_3__website',
 'bedrooms_adj_3__one',
 'bedrooms_adj_3__two',
 'bedrooms_adj_3__three',
 'bedrooms_adj_3__four',
 'bedrooms_adj_3__five',
 'bedrooms_adj_3__six',
 'bedrooms_adj_3__seven',
 'bedrooms_adj_3__Doorman',
 'bedrooms_adj_3__Dogs_Allowed',
 'bedrooms_adj_3__Cats_Allowed',
 'bedrooms_adj_3__Pets_Allowed',
 'bedrooms_adj_3__Elevator',
 'bedrooms_adj_3__Balcony',
 'bedrooms_adj_3__Outdoors',
 'bedrooms_adj_3__Hardwood',
 'bedrooms_adj_3__Internet_Mention',
 'bedrooms_adj_3__Loft',
 'bedrooms_adj_3__New',
 'bedrooms_adj_3__No_fee',
 'bedrooms_adj_3__Gym',
 'bedrooms_adj_3__Parking',
 'bedrooms_adj_3__Storage',
 'bedrooms_adj_3__Sunlight',
 'bedrooms_adj_3__Pool',
 'bedrooms_adj_3__Laundry_Room',
 'bedrooms_adj_3__Wash_Dry',
 'bedrooms_adj_3__Dishwasher',
 'bedrooms_adj_3__Prewar',
 'bedrooms_adj_3__AirCon',
 'mgmt_list_count__website',
 'mgmt_list_count__one',
 'mgmt_list_count__two',
 'mgmt_list_count__three',
 'mgmt_list_count__four',
 'mgmt_list_count__five',
 'mgmt_list_count__six',
 'mgmt_list_count__seven',
 'mgmt_list_count__Doorman',
 'mgmt_list_count__Dogs_Allowed',
 'mgmt_list_count__Cats_Allowed',
 'mgmt_list_count__Pets_Allowed',
 'mgmt_list_count__Elevator',
 'mgmt_list_count__Balcony',
 'mgmt_list_count__Outdoors',
 'mgmt_list_count__Hardwood',
 'mgmt_list_count__Internet_Mention',
 'mgmt_list_count__Loft',
 'mgmt_list_count__New',
 'mgmt_list_count__No_fee',
 'mgmt_list_count__Gym',
 'mgmt_list_count__Parking',
 'mgmt_list_count__Storage',
 'mgmt_list_count__Sunlight',
 'mgmt_list_count__Pool',
 'mgmt_list_count__Laundry_Room',
 'mgmt_list_count__Wash_Dry',
 'mgmt_list_count__Dishwasher',
 'mgmt_list_count__Prewar',
 'mgmt_list_count__AirCon',
 'feature_count__website',
 'feature_count__one',
 'feature_count__two',
 'feature_count__three',
 'feature_count__four',
 'feature_count__five',
 'feature_count__six',
 'feature_count__seven',
 'feature_count__Doorman',
 'feature_count__Dogs_Allowed',
 'feature_count__Cats_Allowed',
 'feature_count__Pets_Allowed',
 'feature_count__Elevator',
 'feature_count__Balcony',
 'feature_count__Outdoors',
 'feature_count__Hardwood',
 'feature_count__Internet_Mention',
 'feature_count__Loft',
 'feature_count__New',
 'feature_count__No_fee',
 'feature_count__Gym',
 'feature_count__Parking',
 'feature_count__Storage',
 'feature_count__Sunlight',
 'feature_count__Pool',
 'feature_count__Laundry_Room',
 'feature_count__Wash_Dry',
 'feature_count__Dishwasher',
 'feature_count__Prewar',
 'feature_count__AirCon',
 'descript_len__website',
 'descript_len__one',
 'descript_len__two',
 'descript_len__three',
 'descript_len__four',
 'descript_len__five',
 'descript_len__six',
 'descript_len__seven',
 'descript_len__Doorman',
 'descript_len__Dogs_Allowed',
 'descript_len__Cats_Allowed',
 'descript_len__Pets_Allowed',
 'descript_len__Elevator',
 'descript_len__Balcony',
 'descript_len__Outdoors',
 'descript_len__Hardwood',
 'descript_len__Internet_Mention',
 'descript_len__Loft',
 'descript_len__New',
 'descript_len__No_fee',
 'descript_len__Gym',
 'descript_len__Parking',
 'descript_len__Storage',
 'descript_len__Sunlight',
 'descript_len__Pool',
 'descript_len__Laundry_Room',
 'descript_len__Wash_Dry',
 'descript_len__Dishwasher',
 'descript_len__Prewar',
 'descript_len__AirCon',
 'photo_count__website',
 'photo_count__one',
 'photo_count__two',
 'photo_count__three',
 'photo_count__four',
 'photo_count__five',
 'photo_count__six',
 'photo_count__seven',
 'photo_count__Doorman',
 'photo_count__Dogs_Allowed',
 'photo_count__Cats_Allowed',
 'photo_count__Pets_Allowed',
 'photo_count__Elevator',
 'photo_count__Balcony',
 'photo_count__Outdoors',
 'photo_count__Hardwood',
 'photo_count__Internet_Mention',
 'photo_count__Loft',
 'photo_count__New',
 'photo_count__No_fee',
 'photo_count__Gym',
 'photo_count__Parking',
 'photo_count__Storage',
 'photo_count__Sunlight',
 'photo_count__Pool',
 'photo_count__Laundry_Room',
 'photo_count__Wash_Dry',
 'photo_count__Dishwasher',
 'photo_count__Prewar',
 'photo_count__AirCon',
 'bedrooms__website',
 'bedrooms__one',
 'bedrooms__two',
 'bedrooms__three',
 'bedrooms__four',
 'bedrooms__five',
 'bedrooms__six',
 'bedrooms__seven',
 'bedrooms__Doorman',
 'bedrooms__Dogs_Allowed',
 'bedrooms__Cats_Allowed',
 'bedrooms__Pets_Allowed',
 'bedrooms__Elevator',
 'bedrooms__Balcony',
 'bedrooms__Outdoors',
 'bedrooms__Hardwood',
 'bedrooms__Internet_Mention',
 'bedrooms__Loft',
 'bedrooms__New',
 'bedrooms__No_fee',
 'bedrooms__Gym',
 'bedrooms__Parking',
 'bedrooms__Storage',
 'bedrooms__Sunlight',
 'bedrooms__Pool',
 'bedrooms__Laundry_Room',
 'bedrooms__Wash_Dry',
 'bedrooms__Dishwasher',
 'bedrooms__Prewar',
 'bedrooms__AirCon',
 'bathrooms__website',
 'bathrooms__one',
 'bathrooms__two',
 'bathrooms__three',
 'bathrooms__four',
 'bathrooms__five',
 'bathrooms__six',
 'bathrooms__seven',
 'bathrooms__Doorman',
 'bathrooms__Dogs_Allowed',
 'bathrooms__Cats_Allowed',
 'bathrooms__Pets_Allowed',
 'bathrooms__Elevator',
 'bathrooms__Balcony',
 'bathrooms__Outdoors',
 'bathrooms__Hardwood',
 'bathrooms__Internet_Mention',
 'bathrooms__Loft',
 'bathrooms__New',
 'bathrooms__No_fee',
 'bathrooms__Gym',
 'bathrooms__Parking',
 'bathrooms__Storage',
 'bathrooms__Sunlight',
 'bathrooms__Pool',
 'bathrooms__Laundry_Room',
 'bathrooms__Wash_Dry',
 'bathrooms__Dishwasher',
 'bathrooms__Prewar',
 'bathrooms__AirCon',
 'log_price__website',
 'log_price__one',
 'log_price__two',
 'log_price__three',
 'log_price__four',
 'log_price__five',
 'log_price__six',
 'log_price__seven',
 'log_price__Doorman',
 'log_price__Dogs_Allowed',
 'log_price__Cats_Allowed',
 'log_price__Pets_Allowed',
 'log_price__Elevator',
 'log_price__Balcony',
 'log_price__Outdoors',
 'log_price__Hardwood',
 'log_price__Internet_Mention',
 'log_price__Loft',
 'log_price__New',
 'log_price__No_fee',
 'log_price__Gym',
 'log_price__Parking',
 'log_price__Storage',
 'log_price__Sunlight',
 'log_price__Pool',
 'log_price__Laundry_Room',
 'log_price__Wash_Dry',
 'log_price__Dishwasher',
 'log_price__Prewar',
 'log_price__AirCon',
 'log_price_sq__website',
 'log_price_sq__one',
 'log_price_sq__two',
 'log_price_sq__three',
 'log_price_sq__four',
 'log_price_sq__five',
 'log_price_sq__six',
 'log_price_sq__seven',
 'log_price_sq__Doorman',
 'log_price_sq__Dogs_Allowed',
 'log_price_sq__Cats_Allowed',
 'log_price_sq__Pets_Allowed',
 'log_price_sq__Elevator',
 'log_price_sq__Balcony',
 'log_price_sq__Outdoors',
 'log_price_sq__Hardwood',
 'log_price_sq__Internet_Mention',
 'log_price_sq__Loft',
 'log_price_sq__New',
 'log_price_sq__No_fee',
 'log_price_sq__Gym',
 'log_price_sq__Parking',
 'log_price_sq__Storage',
 'log_price_sq__Sunlight',
 'log_price_sq__Pool',
 'log_price_sq__Laundry_Room',
 'log_price_sq__Wash_Dry',
 'log_price_sq__Dishwasher',
 'log_price_sq__Prewar',
 'log_price_sq__AirCon',
 'log_price__interest_adj_13__website',
 'log_price__interest_adj_13__one',
 'log_price__interest_adj_13__two',
 'log_price__interest_adj_13__three',
 'log_price__interest_adj_13__four',
 'log_price__interest_adj_13__five',
 'log_price__interest_adj_13__six',
 'log_price__interest_adj_13__seven',
 'log_price__interest_adj_13__Doorman',
 'log_price__interest_adj_13__Dogs_Allowed',
 'log_price__interest_adj_13__Cats_Allowed',
 'log_price__interest_adj_13__Pets_Allowed',
 'log_price__interest_adj_13__Elevator',
 'log_price__interest_adj_13__Balcony',
 'log_price__interest_adj_13__Outdoors',
 'log_price__interest_adj_13__Hardwood',
 'log_price__interest_adj_13__Internet_Mention',
 'log_price__interest_adj_13__Loft',
 'log_price__interest_adj_13__New',
 'log_price__interest_adj_13__No_fee',
 'log_price__interest_adj_13__Gym',
 'log_price__interest_adj_13__Parking',
 'log_price__interest_adj_13__Storage',
 'log_price__interest_adj_13__Sunlight',
 'log_price__interest_adj_13__Pool',
 'log_price__interest_adj_13__Laundry_Room',
 'log_price__interest_adj_13__Wash_Dry',
 'log_price__interest_adj_13__Dishwasher',
 'log_price__interest_adj_13__Prewar',
 'log_price__interest_adj_13__AirCon',
 'log_days_between__website',
 'log_days_between__one',
 'log_days_between__two',
 'log_days_between__three',
 'log_days_between__four',
 'log_days_between__five',
 'log_days_between__six',
 'log_days_between__seven',
 'log_days_between__Doorman',
 'log_days_between__Dogs_Allowed',
 'log_days_between__Cats_Allowed',
 'log_days_between__Pets_Allowed',
 'log_days_between__Elevator',
 'log_days_between__Balcony',
 'log_days_between__Outdoors',
 'log_days_between__Hardwood',
 'log_days_between__Internet_Mention',
 'log_days_between__Loft',
 'log_days_between__New',
 'log_days_between__No_fee',
 'log_days_between__Gym',
 'log_days_between__Parking',
 'log_days_between__Storage',
 'log_days_between__Sunlight',
 'log_days_between__Pool',
 'log_days_between__Laundry_Room',
 'log_days_between__Wash_Dry',
 'log_days_between__Dishwasher',
 'log_days_between__Prewar',
 'log_days_between__AirCon',
 'interest_adj_1',
 'interest_adj_2',
 'interest_adj_3',
 'interest_adj_4',
 'interest_adj_5',
 'interest_adj_6',
 'interest_adj_7',
 'interest_adj_8',
 'interest_adj_9',
 'interest_adj_10',
 'interest_adj_11',
 'interest_adj_12',
 'interest_adj_14',
 'interest_adj_15',
 'interest_adj_16',
 'interest_adj_17',
 'interest_adj_18',
 'interest_adj_19',
 'interest_adj_20',
 'interest_adj_21',
 'interest_adj_22',
 'interest_adj_23',
 'interest_adj_24',
 'interest_adj_25',
 'interest_adj_26',
 'interest_adj_27',
 'interest_adj_28',
 'interest_adj_29',
 'interest_adj_30',
 'distance_adj_1',
 'distance_adj_2',
 'distance_adj_3',
 'distance_adj_4',
 'distance_adj_5',
 'distance_adj_6',
 'distance_adj_7',
 'distance_adj_8',
 'distance_adj_9',
 'distance_adj_10',
 'distance_adj_11',
 'distance_adj_12',
 'distance_adj_13',
 'distance_adj_14',
 'distance_adj_15',
 'distance_adj_16',
 'distance_adj_17',
 'distance_adj_18',
 'distance_adj_19',
 'distance_adj_20',
 'distance_adj_21',
 'distance_adj_22',
 'distance_adj_23',
 'distance_adj_24',
 'distance_adj_25',
 'distance_adj_26',
 'distance_adj_27',
 'distance_adj_28',
 'distance_adj_29',
 'distance_adj_30',
 'days_between_adj_1',
 'days_between_adj_2',
 'days_between_adj_3',
 'days_between_adj_4',
 'days_between_adj_6',
 'days_between_adj_7',
 'days_between_adj_8',
 'days_between_adj_9',
 'days_between_adj_10',
 'days_between_adj_11',
 'days_between_adj_12',
 'days_between_adj_13',
 'days_between_adj_14',
 'days_between_adj_15',
 'days_between_adj_16',
 'days_between_adj_17',
 'days_between_adj_18',
 'days_between_adj_19',
 'days_between_adj_20',
 'days_between_adj_21',
 'days_between_adj_22',
 'days_between_adj_23',
 'days_between_adj_24',
 'days_between_adj_25',
 'days_between_adj_26',
 'days_between_adj_27',
 'days_between_adj_28',
 'days_between_adj_29',
 'days_between_adj_30',
 'feature_count_adj_1',
 'feature_count_adj_2',
 'feature_count_adj_3',
 'feature_count_adj_4',
 'feature_count_adj_5',
 'feature_count_adj_6',
 'feature_count_adj_7',
 'feature_count_adj_8',
 'feature_count_adj_9',
 'feature_count_adj_10',
 'feature_count_adj_11',
 'feature_count_adj_12',
 'feature_count_adj_13',
 'feature_count_adj_14',
 'feature_count_adj_15',
 'feature_count_adj_16',
 'feature_count_adj_17',
 'feature_count_adj_18',
 'feature_count_adj_19',
 'feature_count_adj_20',
 'feature_count_adj_21',
 'feature_count_adj_22',
 'feature_count_adj_23',
 'feature_count_adj_24',
 'feature_count_adj_26',
 'feature_count_adj_27',
 'feature_count_adj_28',
 'feature_count_adj_29',
 'feature_count_adj_30',
 'descript_len_adj_1',
 'descript_len_adj_2',
 'descript_len_adj_3',
 'descript_len_adj_4',
 'descript_len_adj_5',
 'descript_len_adj_6',
 'descript_len_adj_7',
 'descript_len_adj_8',
 'descript_len_adj_9',
 'descript_len_adj_10',
 'descript_len_adj_12',
 'descript_len_adj_13',
 'descript_len_adj_14',
 'descript_len_adj_15',
 'descript_len_adj_16',
 'descript_len_adj_17',
 'descript_len_adj_18',
 'descript_len_adj_19',
 'descript_len_adj_20',
 'descript_len_adj_21',
 'descript_len_adj_22',
 'descript_len_adj_23',
 'descript_len_adj_24',
 'descript_len_adj_25',
 'descript_len_adj_26',
 'descript_len_adj_27',
 'descript_len_adj_28',
 'descript_len_adj_29',
 'descript_len_adj_30',
 'photo_count_adj_1',
 'photo_count_adj_2',
 'photo_count_adj_3',
 'photo_count_adj_4',
 'photo_count_adj_5',
 'photo_count_adj_6',
 'photo_count_adj_7',
 'photo_count_adj_8',
 'photo_count_adj_9',
 'photo_count_adj_10',
 'photo_count_adj_11',
 'photo_count_adj_12',
 'photo_count_adj_13',
 'photo_count_adj_14',
 'photo_count_adj_15',
 'photo_count_adj_16',
 'photo_count_adj_17',
 'photo_count_adj_18',
 'photo_count_adj_19',
 'photo_count_adj_20',
 'photo_count_adj_21',
 'photo_count_adj_22',
 'photo_count_adj_23',
 'photo_count_adj_24',
 'photo_count_adj_25',
 'photo_count_adj_26',
 'photo_count_adj_27',
 'photo_count_adj_28',
 'photo_count_adj_29',
 'photo_count_adj_30',
 'log_price_adj_1',
 'log_price_adj_2',
 'log_price_adj_4',
 'log_price_adj_5',
 'log_price_adj_6',
 'log_price_adj_7',
 'log_price_adj_8',
 'log_price_adj_9',
 'log_price_adj_10',
 'log_price_adj_11',
 'log_price_adj_12',
 'log_price_adj_13',
 'log_price_adj_14',
 'log_price_adj_15',
 'log_price_adj_16',
 'log_price_adj_17',
 'log_price_adj_18',
 'log_price_adj_19',
 'log_price_adj_20',
 'log_price_adj_21',
 'log_price_adj_22',
 'log_price_adj_23',
 'log_price_adj_24',
 'log_price_adj_25',
 'log_price_adj_26',
 'log_price_adj_27',
 'log_price_adj_28',
 'log_price_adj_29',
 'log_price_adj_30',
 'bedrooms_adj_1',
 'bedrooms_adj_2',
 'bedrooms_adj_4',
 'bedrooms_adj_5',
 'bedrooms_adj_6',
 'bedrooms_adj_7',
 'bedrooms_adj_8',
 'bedrooms_adj_9',
 'bedrooms_adj_10',
 'bedrooms_adj_11',
 'bedrooms_adj_12',
 'bedrooms_adj_13',
 'bedrooms_adj_14',
 'bedrooms_adj_15',
 'bedrooms_adj_16',
 'bedrooms_adj_17',
 'bedrooms_adj_18',
 'bedrooms_adj_19',
 'bedrooms_adj_20',
 'bedrooms_adj_21',
 'bedrooms_adj_22',
 'bedrooms_adj_23',
 'bedrooms_adj_24',
 'bedrooms_adj_25',
 'bedrooms_adj_26',
 'bedrooms_adj_27',
 'bedrooms_adj_28',
 'bedrooms_adj_29',
 'bedrooms_adj_30',
 'bathrooms_adj_1',
 'bathrooms_adj_2',
 'bathrooms_adj_3',
 'bathrooms_adj_5',
 'bathrooms_adj_6',
 'bathrooms_adj_7',
 'bathrooms_adj_8',
 'bathrooms_adj_9',
 'bathrooms_adj_10',
 'bathrooms_adj_11',
 'bathrooms_adj_12',
 'bathrooms_adj_13',
 'bathrooms_adj_14',
 'bathrooms_adj_15',
 'bathrooms_adj_16',
 'bathrooms_adj_17',
 'bathrooms_adj_18',
 'bathrooms_adj_19',
 'bathrooms_adj_20',
 'bathrooms_adj_21',
 'bathrooms_adj_22',
 'bathrooms_adj_23',
 'bathrooms_adj_24',
 'bathrooms_adj_25',
 'bathrooms_adj_26',
 'bathrooms_adj_27',
 'bathrooms_adj_28',
 'bathrooms_adj_29',
 'bathrooms_adj_30']

scale = CustomScaler(columns=continuous_cols2 )

train_all_scale = scale.fit_transform(train_all)
holdout_scale = scale.fit_transform(holdout)  

In [114]:
#convert from pandas to numpy arrays and take out the id columns at the beginning

#For the whole dataset: 
mat_train = train_all_scale.as_matrix()
X_train_all = mat_train[:,3:]
y_train_all = mat_train[:, 2]
mat_holdout = holdout_scale.as_matrix()
X_holdout = mat_holdout[:,3:]

#For the small data:
mat_train_small = train_all_scale_small.as_matrix()
X_train_all_small = mat_train_small[:,3:61]
y_train_all_small = mat_train_small[:, 2]
mat_holdout_small = holdout_scale_small.as_matrix()
X_holdout_small = mat_holdout_small[:,3:61]

In [102]:
#convert the location column using one-hot encoding: 

#since this creates a lot of dummy variables, another way to do it would be to add a column for averge interest level for that
#location code 

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(train_all_small["location_code"].reshape(-1,1))
loc_codes_train = enc.transform(train_all_small["location_code"].reshape(-1,1)).toarray()
#change for 0/1 encoding to -1/1 encoding: 
loc_codes_train[loc_codes_train ==0 ] = -1
#add these new columns to the data array: 
X_train_all_small = np.concatenate((X_train_all_small, loc_codes_train),axis =1)
#delete the 16th location because there is no location 16 in the test set
X_train_all_small = X_train_all_small[:, :74]

#do this for the holdout data too: 
enc.fit(holdout_small["location_code"].reshape(-1,1))
loc_codes_holdout = enc.transform(holdout_small["location_code"].reshape(-1,1)).toarray()
#change for 0/1 encoding to -1/1 encoding: 
loc_codes_holdout[loc_codes_holdout ==0 ] = -1
#add these new columns to the data array: 
X_holdout_small = np.concatenate((X_holdout_small, loc_codes_holdout),axis =1)

In [115]:
#One-hot encoding for the full dataset: 

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(train_all["location_code"].reshape(-1,1))
loc_codes_train = enc.transform(train_all["location_code"].reshape(-1,1)).toarray()
#change for 0/1 encoding to -1/1 encoding: 
loc_codes_train[loc_codes_train ==0 ] = -1
#add these new columns to the data array: 
X_train_all = np.concatenate((X_train_all, loc_codes_train),axis =1)
#delete the 16th location because there is no location 16 in the test set
X_train_all = X_train_all[:, :837]

#do this for the holdout data too: 
enc.fit(holdout["location_code"].reshape(-1,1))
loc_codes_holdout = enc.transform(holdout["location_code"].reshape(-1,1)).toarray()
#change for 0/1 encoding to -1/1 encoding: 
loc_codes_holdout[loc_codes_holdout ==0 ] = -1
#add these new columns to the data array: 
X_holdout = np.concatenate((X_holdout, loc_codes_holdout),axis =1)

In [116]:
print(loc_codes_train.shape)
print(X_train_all_small.shape)
print(loc_codes_holdout.shape)
print(X_holdout_small.shape)
print()
print(X_holdout.shape)
print(X_train_all.shape)

(49337, 17)
(49337, 58)
(74659, 16)
(74659, 58)

(74659, 837)
(49337, 837)


In [22]:
# test/train split (full dataset):   

from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X_train_all, y_train_all)    

In [23]:
# test/train split (small dataset):

from sklearn.model_selection import train_test_split 
X_train_small, X_test_small, y_train_small, y_test = train_test_split(X_train_all_small, y_train_all_small)    

In [482]:
#full dataset: 
print(X_train.shape)
print(X_test.shape)
print(train_all.shape)

#for the small dataset:
print(X_train_no.shape)
print(X_test_no.shape)
print(X_train_all_scale_small.shape)

(37014, 821)
(12338, 821)
(49352, 824)
(37014, 62)
(12338, 62)
(49337, 62)


#### Baseline (Random) Classifier Performance:

In [112]:
per_y1 = len(y_train[y_train== 1])/len(y_train)
per_y2 = len(y_train[y_train== 2])/len(y_train)
per_y3 = len(y_train[y_train== 3])/len(y_train)

y1 = len(y_train[y_train== 1])
y2 = len(y_train[y_train== 2])
y3 = len(y_train[y_train== 3])

print("baseline log loss:")
- (1/X_train.shape[0])*(np.log(per_y1)*y1  + np.log(per_y2)*y2 + np.log(per_y3)*y3)


baseline log loss:


0.78391113651128785

log loss on the holdout set predicting all 1's = 10.63995

# Multi Layer Perceptron Neural Net:

##### Multi-layer Perceptrons are:
- The essential deep-learning model. We'd use an RNN for data with a time component and a CNN for image data (that's highly correlated between the pixel data points). But for a problem like this, a "basic" MLP is the way to go. 
- MLP's are also called deep feed-forward neural networks 
- Their goal is to learn the function mapping the input data to the classification labels (real function called f-star in the book. The approximation is just regular f(x))
- Capability to learn non-linear models.
- RNNs are the neural nets that can feed back to itself. 
- Neural networks are called networks because they are a chain of non-linear function compositions f3 (f2 (f1(x))). f1 is the first layer. f2 second layer, etc. 

##### Why Neural Nets are more powerful than linear models: 
- Linear and logistic regression are limited to linear functions, so the model cannot understand the interaction between any two input variables (without having to put those interaction explicitly into the model). 
- You don't have to transform any variables when using NN's because NN's are already transforming the predictors with nonlinear functions 
- However, the weights aren't interpretable like in logistic regression 
- Too many hidden units leads to overfitting 

##### The disadvantages of Multi-layer Perceptron are:
- MLP with hidden layers have a non-convex loss function where there exists more than one local minimum. Therefore different random weight initializations can lead to different validation accuracy.
- MLP requires tuning a number of hyperparameters such as the number of hidden neurons, layers, and iterations.
- MLP is sensitive to feature scaling.

##### How it works: 
- MLP trains using Backpropagation. 
- More precisely, it trains using some form of gradient descent and the gradients are calculated using Backpropagation. 
- For classification, it minimizes the Cross-Entropy loss function.

##### Time complexity:
Suppose there are n training samples, m features, k hidden layers, each containing h neurons - for simplicity, and o output neurons. The time complexity of backpropagation is $O(n \cdot m \cdot h^k \cdot o \cdot i) $, where i is the number of iterations.

##### About the parameters: 
- Alpha: 
 - (L2 regularization) term which helps in avoiding overfitting by penalizing weights with large magnitudes.
- Activation Function for the output layer:
 - Softmax is the necessary choice so that no matter what values are plugged into predict_proba(), the output probability vector always sums up to 1. This can only be achieved by the Softmax activation function. Using an activation other that Softmax there is no guarantee that the sum of the activations in the final layer will be exactly one. 
- Activation Functions in the hidden layers:
 - tanh works better than logistic because the tanh function is symmetric about 0 so it is less prone to "saturation in the later layers". It also ensures that the input, output and hidden layers all have mean values of 0 and stand dev = 1. 
 - ReLu has been shown to reach the same training error 6 times faster than tanh. However, relu isn't differentiable at zero and so it doesn't train a unit when it's "zero active". Leaky Relu and PReLu have been proposed to solve this problem but aren't implemented in sklearn yet. 
- Solvers: 
  - Empirically, we observed that L-BFGS converges faster and with better solutions on small datasets. 
  - For relatively large datasets, however, Adam is very robust. It usually converges quickly and gives pretty good performance. 
  - SGD with momentum or nesterov’s momentum, on the other hand, can perform better than those two algorithms if learning rate is correctly tuned.
- Hidden Layers:
  - hidden_layer_sizes=(5, 3) means there are 2 hidden layers, 5 hidden units in first layer and 3 hidden units in second layer.

### Fit the MLP classifier using Grid Search: 

About GridSearchCV function:
- by default does 3-fold cross validation 
- also does stratified Kfold by default, which preserves the percentage of samples for each class
- use scoring = neg_log_loss (so it knows that lower is better)

In [158]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import log_loss

gs = GridSearchCV (  
    
        MLPClassifier(),
                  
        param_grid={
        #'learning_rate': [0.005, 0.001],    <- only when solver is 'sgd'
        'alpha': 10.0 ** -np.arange(1, 7),
        'hidden_layer_sizes': [(5,), (4,), (3,), (6,), ],
        'solver': ['adam', 'lbfgs','sgd'],
        'activation': ["relu", "tanh"]
         }, 
                  
        scoring = 'neg_log_loss' )         


In [ ]:
#------------ Other things to try in grid search: -------------------------

#MLPClassifier(solver='adam', random_state=1, activation = "tanh")    <- optimized for the old dataset (with all the correlations)
        #"random_state" = 1,
    #'hidden0__units': [4, 8, 12],
    #'activation': ["relu", "logistic", "tanh"],
        #'learning_rate': [0.005, 0.001],
        #hidden_layer_sizes=(5, 2)
        #'hidden_layer_sizes': [(5, 2), (5,), (10,), (2,), (7,)]

In [ ]:
gs.fit(X_train,y_train)

#### Exploring the best results....

In [280]:
from sklearn.metrics import classification_report

print("Best parameters set found on development set:")
print()
print(gs.best_params_)
print()
print("Best Mean log loss on the kth fold during training:")
print(sorted(-1* gs.cv_results_['mean_test_score'])[0])
print("Best Mean log loss on the training folds during training:")
print(sorted(-1* gs.cv_results_['mean_train_score'])[0])
print("Mean log loss on the training folds for the test set's best set of parameters")
print(-1* gs.cv_results_['mean_train_score'][np.argsort(-1* gs.cv_results_['mean_test_score'])][0])
print()

#print("All mean log losses testing (using 3 fold CV) :")
means_orig = -1 * gs.cv_results_['mean_test_score']
stds  = gs.cv_results_['std_test_score']

means = sorted(means_orig)
order = np.argsort(means_orig)
stds = stds[order]
param_list = gs.cv_results_['params']  

print("top means:", means[0:5])
print("  and their std devs:", stds[0:5])

Best parameters set found on development set:

{'activation': 'tanh', 'solver': 'sgd', 'alpha': 1.0000000000000001e-05, 'hidden_layer_sizes': (4,)}

Best Mean log loss on the kth fold during training:
0.624649655765
Best Mean log loss on the training folds during training:
0.531438061999
Mean log loss on the training folds for the test set's best set of parameters
0.590233290831

top means: [0.62464965576462683, 0.62507566115723756, 0.6255623511494266, 0.62564019637260337, 0.62565603401020642]
  and their std devs: [ 0.00218244  0.00185456  0.00070539  0.00321176  0.00179811]


#### Statistically significant best results?

In [247]:
import scipy 
from scipy import stats

print("T tests.....")

def calculate_t_stat(index1, index2):
    t_stat = (means[index1] - means[index2] - 0)/(np.sqrt(stds[index1])**2/3 + np.sqrt(stds[index2])**2/3)
    return t_stat

t_stat = calculate_t_stat(0,25)
p_val =  stats.t.sf(np.abs(t_stat), 3-1)*2     #divide by 2 for 1-tailed test? 

print("t-statistic & p-value for testing whether the differece in mean log losses is significantly different from 0: ") 
print("       ", t_stat,  p_val)

T tests.....
t-statistic & p-value for testing whether the differece in mean log losses is significantly different from 0: 
        -1.64148905887 0.242393044984


In [289]:
print("Best sets of parameters:")
print()

for i in range(10):         #change to range(len(mean))   to see the whole list
    t_stat = calculate_t_stat(0,i)
    print("Distance from best set:", i)
    print("parameters:", gs.cv_results_['params'][order[i]])
    print("T-stat that these params are better than the best set:", t_stat, "and p-value:", stats.t.sf(np.abs(t_stat), 3-1)*2 )
    print()

Best sets of parameters:

Distance from best set: 0
parameters: {'activation': 'tanh', 'solver': 'sgd', 'alpha': 1.0000000000000001e-05, 'hidden_layer_sizes': (4,)}
T-stat that these params are better than the best set: 0.0 and p-value: 1.0

Distance from best set: 1
parameters: {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'hidden_layer_sizes': (5,)}
T-stat that these params are better than the best set: -0.316575190986 and p-value: 0.781553798158

Distance from best set: 2
parameters: {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.01, 'hidden_layer_sizes': (4,)}
T-stat that these params are better than the best set: -0.948144268788 and p-value: 0.443132338995

Distance from best set: 3
parameters: {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.001, 'hidden_layer_sizes': (6,)}
T-stat that these params are better than the best set: -0.550891232703 and p-value: 0.637027661561

Distance from best set: 4
parameters: {'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.1000

These results show that....
 - we can stop trying relu (tanh is better)
 - alpha doesn't matter much 
 - sgd is the best solver (start adding learning rate to the parameters list)
 - number of units in first hidden layer doesn't appear to matter much either 

#### Fit on the best set of parameters: 

In [266]:
gs.cv_results_['params'][order[0]]

{'activation': 'tanh',
 'alpha': 1.0000000000000001e-05,
 'hidden_layer_sizes': (4,),
 'solver': 'sgd'}

In [341]:
# classifier with the best parameters from grid search: 
clf = MLPClassifier(activation = gs.cv_results_['params'][order[0]]["activation"], 
                    alpha = gs.cv_results_['params'][order[0]]["alpha"],
                    hidden_layer_sizes = gs.cv_results_['params'][order[0]]["hidden_layer_sizes"],
                    solver = gs.cv_results_['params'][order[0]]["solver"] )

AttributeError: 'GridSearchCV' object has no attribute 'cv_results_'

In [343]:
clf.fit(X_train,y_train)
y_pred2 = clf.predict(X_test)
y_pred_proba2 = clf.predict_proba(X_test)

In [344]:
print("log loss from the best set of grid search parameters on my test set:")
log_loss(y_test, y_pred_proba2)  

log loss from the best set of grid search parameters on my test set:


0.62584323078537163

Predicted class distribution of the test set (as a sanity check): 

In [345]:
print("percentage of Lows: ", len(y_pred2[y_pred2 == 1])/len(y_pred2) * 100)
print("percentage of Mediums: ", len(y_pred2[y_pred2 == 2])/len(y_pred2) * 100)
print("percentage of Highs: ", len(y_pred2[y_pred2 == 3])/len(y_pred2)* 100)

print()
print("Number of mislabeled points out of a total %d points : %d"
       % (X_test.shape[0],(y_test != y_pred2).sum()))

print()
print("percent accuracy: ", clf.score(X_test,y_test) )

percentage of Lows:  80.48306046360837
percentage of Mediums:  16.315448208785867
percentage of Highs:  3.201491327605771

Number of mislabeled points out of a total 12338 points : 3437

percent accuracy:  0.721429729292


In [273]:
#clf.coefs_ contains the weight matrices that constitute the model parameters:
[coef.shape for coef in clf.coefs_]

[(823, 4), (4, 3)]

## Next grid search attempt:

In [ ]:
gs2 = GridSearchCV (  
    
        MLPClassifier(activation = 'tanh', solver = 'sgd'),
                  
        param_grid={
        #'learning_rate': ['constant', 'invscaling', 'adaptive'],    #<- only when solver is 'sgd'
        'alpha': 10.0 ** -np.arange(1, 7),
        'hidden_layer_sizes': [(5,), (4,), (3,), (6,), (4,2)],
         }, 
                  
        scoring = 'neg_log_loss' )         

In [ ]:
gs2.fit(X_train_small,y_train_small)

## Trying different datasets (move this to the top) 
Is it better to use all columns OR just the ones without interactions OR the ones without adjacency and interactions? 
(cuz neural nets automatically build the interaction terms):

In [52]:
#fit on a just the first few columns: 

clf.fit(X_train[:,0:47],y_train)
y_pred = clf.predict(X_test[:,0:47])
y_pred_proba = clf.predict_proba(X_test[:,0:47])
y_pred_proba
from sklearn.metrics import log_loss
log_loss(y_test, y_pred_proba)   

0.61760745647523108

In [49]:
#fit on all columns: 

#clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
y_pred_proba = clf.predict_proba(X_test)
y_pred_proba
from sklearn.metrics import log_loss
log_loss(y_test, y_pred_proba)   

0.61062277319002833

In [50]:
#fit on all non-interaction columns: 

indices = list(range(0,47))   
indices.extend(list(range(560, 823))) 

clf.fit(X_train[:,indices],y_train)
y_pred = clf.predict(X_test[:,indices])
y_pred_proba = clf.predict_proba(X_test[:,indices])
y_pred_proba
from sklearn.metrics import log_loss
log_loss(y_test, y_pred_proba)   

0.61891664795650025

In [387]:
X_train.shape

(37014, 821)

In [122]:
#fit using no interactions and only the adjacencies that aren't higly correlated: 

clf.fit(X_train_no,y_train)
y_pred_proba = clf.predict_proba(X_test_no)
from sklearn.metrics import log_loss
log_loss(y_test, y_pred_proba) 

0.000882497002522934

#### Using cross validation:

Outputs the log loss on a test test set k times. 

In [72]:
from sklearn.model_selection import cross_val_score

#Stratigied K-fold perserves the number of examples from each class 
cv_scores = cross_val_score(clf, X_train_all_scale[:,indices], y_train_all, scoring='neg_log_loss', cv = 10) 
cv_scores

array([-0.62436891, -0.6084782 , -0.61548945, -0.59919801, -0.6109528 ,
       -0.62454158, -0.61820943, -0.6302879 , -0.62648001, -0.63441894])

In [73]:
print("mean log loss:", -1*np.mean(cv_scores))

mean log loss: 0.619242524111


In [76]:
print("standard dev of cross val log losses :",np.std(cv_scores))

standard dev of cross val log losses : 0.01029849946


In [ ]:
#mean log loss: 0.620394160421    all columns 
#mean log loss: 0.619242524111     minus interactions    <--- lowest loss! 
#mean log loss: 0.625933638544     minus interactions and adjacencies

### Storing Old Results: 

logloss = 0.62709157338049504    
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                   hidden_layer_sizes=(5, 2), random_state=1)
---------------------------------------------------------      
logloss = 0.61944758672494526    
clf = MLPClassifier(solver='lbfgs', alpha=1e-2,
                   hidden_layer_sizes=(5, 2), random_state=1)
---------------------------------------------------------    
*Dataset:* no interactions, all adjacencies

{'activation': 'tanh', 'alpha': 0.001, 'hidden_layer_sizes': (5,)}

With a training score of:
-0.624981302943

testing log loss = 0.6271359

Kaggle score:  0.62316

---------------------------------------------------------    
*Dataset:* no interactions, reduced adjacencies

{'activation': 'tanh', 'solver': 'lbfgs', 'alpha': 0.001, 'hidden_layer_sizes': (5,)}

With a training score of:
-0.615696513033

testing log loss = 0.6271359

Kaggle score: 0.62571

---------------------------------------------------------    
*Dataset:* all interactions, all adjacencies 

{'activation': 'tanh', 'solver': 'sgd', 'alpha': 1.0000000000000001e-05, 'hidden_layer_sizes': (4,)}

With a training score of:
-0.624649655765

testing log loss: 0.61738848385994771

Kaggle score: 0.62300

---------------------------------------------------------   
*Dataset:* all interactions, all adjacencies 

{activation ='tanh',alpha =  1.0000000000000001e-05,hidden_layer_sizes = (4,),solver = 'sgd' }

Best Mean log loss on the kth fold during training:
0.624649655765

log loss from the best set of grid search parameters on my test set:
0.62584323078537163

Kaggle score:  0.62556

-------------------------------------------------------------------------
*Dataset*: corrected scaling, small model 

Models details: 

random seed = 7, possibily with cross validation? 
#stop early if testing error gets large: 
callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')

#Create the model: 
model = Sequential()

#Input layer: 
model.add(Dense(50, input_dim= inshape, activation = 'tanh',              
                #kernel_regularizer=regularizers.l2(0.001),
                #activity_regularizer=regularizers.l1(0.001), 
                kernel_initializer= initializers.glorot_normal(seed=None))) 
#model.add(BatchNormalization())
model.add(Dropout(0.5))     

#One hidden layer: 
model.add(Dense(50,  activation = 'tanh',
                #kernel_regularizer=regularizers.l2(0.001),
                #activity_regularizer=regularizers.l1(0.001), 
                kernel_initializer= initializers.glorot_normal(seed=None)))
#model.add(BatchNormalization())
model.add(Dropout(0.25))

#Second hidden layer: 
model.add(Dense(50, activation = 'tanh', 
                #kernel_regularizer=regularizers.l2(0.01),
                #activity_regularizer=regularizers.l1(0.01), 
                kernel_initializer= initializers.glorot_normal(seed=None)))
#model.add(BatchNormalization())
model.add(Dropout(0.25))

#Output layer 
model.add(Dense(3, activation='softmax',
                #kernel_regularizer=regularizers.l2(0.01),
                #activity_regularizer=regularizers.l1(0.01), 
                kernel_initializer= initializers.glorot_normal(seed=None)))

#Setting up to optimize the weights: 
sgd = optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_small, categorical_labels_train , epochs=150, batch_size=10)

My test set loss: 0.60261446516877804

Training loss:0.6401 

Kaggle score: 0.61680

## Make a kaggle submission: 

In [32]:
y_pred_proba_holdout = model.predict_proba(X_holdout_small)

#sanity check (that doesn't work for keras models)
y_pred_holdout = model.predict(X_holdout_small)
print("\n percentage predicted low label:")
print(len(y_pred_holdout[y_pred_holdout== 1])/X_holdout_small.shape[0]) 

y_pred_proba_holdout

74464/74659 [============================>.] - ETA: 0s
 percentage predicted low label:
0.0


array([[ 0.68023771,  0.29222378,  0.02753843],
       [ 0.89006209,  0.09311015,  0.01682781],
       [ 0.97496641,  0.02343635,  0.00159733],
       ..., 
       [ 0.88118148,  0.10383328,  0.01498524],
       [ 0.97141707,  0.02684902,  0.00173388],
       [ 0.61404651,  0.34472221,  0.0412313 ]], dtype=float32)

In [33]:
submission = pd.DataFrame({
        "listing_id": holdout["listing_id"],
        "high": y_pred_proba_holdout[:,2],
        "medium":y_pred_proba_holdout[:,1],
        "low": y_pred_proba_holdout[:,0]
    })

In [561]:
columnsTitles=["listing_id","high","medium","low"]
submission=submission.reindex(columns=columnsTitles)
submission.head()   

,listing_id,high,medium,low
0,7142618,0.038962,0.249934,0.711104
1,7210040,0.023934,0.108455,0.867611
2,7174566,0.001243,0.026781,0.971976
3,7191391,0.134686,0.382049,0.483265
4,7171695,0.012598,0.144559,0.842843


In [34]:
submission.to_csv('submission_keras2.csv', index=False)

# Using Keras

In [28]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Dropout
import numpy
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras import regularizers
from keras import callbacks
from keras.layers.normalization import BatchNormalization
from keras import optimizers, initializers
from keras.utils.np_utils import to_categorical

numpy.random.seed(7)

In [488]:
#Keras needs the labels to be in binary 0/1 vectors for each class: 

categorical_labels_train = to_categorical(y_train_small-1, num_classes=None)
categorical_labels_test = to_categorical(y_test-1, num_classes=None)


About this model:
 - In the input layer you need to specify the number of predictors coming in from the dataset
 - Output layer size should be the number of classes & the activaction should be softmax to get the predic_proba
 - Using dropout helps determine overfitting. 25% dropout means 25% of the hidden units are randomly removed from the model.
 - Batch Normalizatin noramlizes the activations fo the previous layer at each batch , i.e. applies a transformation that maintains the mean activation close to 0 and the activation standard deviation close to 1.
 - Xavier initialization makes sure the weights start off at the right size so they don't shrink through each layer so too small to be useful and so they don't start to grow too massive to be useful. 

In [528]:
#input layer shape:
inshape = X_train_small.shape[1]

#stop early if testing error gets large: 
callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')

#Create the model: 
model = Sequential()

#Input layer: 
model.add(Dense(50, input_dim= inshape, activation = 'tanh',              
                #kernel_regularizer=regularizers.l2(0.001),
                #activity_regularizer=regularizers.l1(0.001), 
                kernel_initializer= initializers.glorot_normal(seed=None))) 
#model.add(BatchNormalization())
model.add(Dropout(0.5))     

#One hidden layer: 
model.add(Dense(50,  activation = 'tanh',
                #kernel_regularizer=regularizers.l2(0.001),
                #activity_regularizer=regularizers.l1(0.001), 
                kernel_initializer= initializers.glorot_normal(seed=None)))
#model.add(BatchNormalization())
model.add(Dropout(0.25))

#Second hidden layer: 
model.add(Dense(50, activation = 'tanh', 
                #kernel_regularizer=regularizers.l2(0.01),
                #activity_regularizer=regularizers.l1(0.01), 
                kernel_initializer= initializers.glorot_normal(seed=None)))
#model.add(BatchNormalization())
model.add(Dropout(0.25))

#Output layer 
model.add(Dense(3, activation='softmax',
                #kernel_regularizer=regularizers.l2(0.01),
                #activity_regularizer=regularizers.l1(0.01), 
                kernel_initializer= initializers.glorot_normal(seed=None)))

#Setting up to optimize the weights: 
sgd = optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [529]:
#Without grid search: 
model.fit(X_train_small, categorical_labels_train , epochs=10, batch_size=10)

Epoch 1/10
37002/37002 [==============================] - 13s - loss: 0.7191 - acc: 0.6925    
Epoch 2/10
37002/37002 [==============================] - 10s - loss: 0.6763 - acc: 0.7041    
Epoch 3/10
37002/37002 [==============================] - 10s - loss: 0.6640 - acc: 0.7059    
Epoch 4/10
37002/37002 [==============================] - 11s - loss: 0.6558 - acc: 0.7101    
Epoch 5/10
37002/37002 [==============================] - 12s - loss: 0.6516 - acc: 0.7143    
Epoch 6/10
37002/37002 [==============================] - 11s - loss: 0.6509 - acc: 0.7124    
Epoch 7/10
37002/37002 [==============================] - 11s - loss: 0.6475 - acc: 0.7133    
Epoch 8/10
37002/37002 [==============================] - 11s - loss: 0.6460 - acc: 0.7133    
Epoch 9/10
37002/37002 [==============================] - 11s - loss: 0.6434 - acc: 0.7143    
Epoch 10/10
37002/37002 [==============================] - 11s - loss: 0.6401 - acc: 0.7149    


In [549]:
#Predict the probabilities on the test set for the Keras model:

print("log loss on the test set:")
proba = model.predict_proba(X_test_small)
log_loss(y_test, proba)

log loss on the test set:
11872/12335 [===========================>..] - ETA: 0s

0.60261446516877804

#### Keras Cross Val Take2:

In [29]:
# define 5-fold cross validation test harness
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
cvscores = []
i=1

for train, test in kfold.split(X_train_all_small, y_train_all_small):
    print("fold number:", i)
    model = Sequential()
    
    inshape = X_train_small.shape[1]

    #Input layer: 
    model.add(Dense(50, input_dim= inshape, activation = 'tanh',              
                #kernel_regularizer=regularizers.l2(0.001),
                #activity_regularizer=regularizers.l1(0.001), 
                kernel_initializer= initializers.glorot_normal(seed=None))) 
    model.add(BatchNormalization())
    model.add(Dropout(0.5))     

    #One hidden layer: 
    model.add(Dense(50,  activation = 'tanh',
                #kernel_regularizer=regularizers.l2(0.001),
                #activity_regularizer=regularizers.l1(0.001), 
                kernel_initializer= initializers.glorot_normal(seed=None)))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    #Second hidden layer: 
    model.add(Dense(50, activation = 'tanh', 
                #kernel_regularizer=regularizers.l2(0.01),
                #activity_regularizer=regularizers.l1(0.01), 
                kernel_initializer= initializers.glorot_normal(seed=None)))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    #Output layer 
    model.add(Dense(3, activation='softmax',
                #kernel_regularizer=regularizers.l2(0.01),
                #activity_regularizer=regularizers.l1(0.01), 
                kernel_initializer= initializers.glorot_normal(seed=None)))

    #Setting up to optimize the weights: 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    
    categorical_labels = to_categorical(y_train_all_small[train]-1, num_classes=None)
    model.fit(X_train_all_small[train],categorical_labels, epochs=150, batch_size=10, verbose=1)
    
    # evaluate the model
    categorical_labels_test = to_categorical(y_train_all_small[test]-1, num_classes=None)
    scores = model.evaluate(X_train_all_small[test], categorical_labels_test, verbose=0)
    print("log loss on test k-th fold:", scores[0])
    cvscores.append(scores[0])
    
    i+=1

print()
print("mean testing log loss:")
print(numpy.mean(cvscores), "+/-", numpy.std(cvscores))

fold number: 1
Epoch 1/150
39469/39469 [==============================] - 8s - loss: 0.7199 - acc: 0.6917     
Epoch 2/150
39469/39469 [==============================] - 7s - loss: 0.6771 - acc: 0.7017     
Epoch 3/150
39469/39469 [==============================] - 7s - loss: 0.6650 - acc: 0.7058     
Epoch 4/150
39469/39469 [==============================] - 7s - loss: 0.6593 - acc: 0.7087     
Epoch 5/150
39469/39469 [==============================] - 7s - loss: 0.6549 - acc: 0.7109     
Epoch 6/150
39469/39469 [==============================] - 7s - loss: 0.6533 - acc: 0.7087     
Epoch 7/150
39469/39469 [==============================] - 7s - loss: 0.6508 - acc: 0.7106     
Epoch 8/150
39469/39469 [==============================] - 8s - loss: 0.6471 - acc: 0.7130     
Epoch 9/150
39469/39469 [==============================] - 7s - loss: 0.6445 - acc: 0.7111     
Epoch 10/150
39469/39469 [==============================] - 7s - loss: 0.6425 - acc: 0.7112     
Epoch 11/150
39469/39469

In [31]:
from sklearn.metrics import log_loss
print("log loss on the test set:")
proba = model.predict_proba(X_test_small)
log_loss(y_test, proba)


log loss on the test set:
10752/12335 [=========================>....] - ETA: 0s

0.58680211244944525

Kaggle score: 0.62309

#### Keras Cross Val Take 3  (Standardizing all the binary colums)

In [47]:
# define 5-fold cross validation test harness
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
cvscores3 = []
i=1

scale_stand = StandardScaler()

small_scaled_bin = scale_stand.fit_transform(X_train_all_small)

for train, test in kfold.split(small_scaled_bin, y_train_all_small):
    print("fold number:", i)
    model3 = Sequential()
    
    inshape = small_scaled_bin.shape[1]

    #Input layer: 
    model3.add(Dense(50, input_dim= inshape, activation = 'tanh',         
                #kernel_regularizer=regularizers.l2(0.001),
                #activity_regularizer=regularizers.l1(0.001), 
                kernel_initializer= initializers.glorot_normal(seed=None)))
                     
    #model3.add(BatchNormalization())
    model3.add(Dropout(0.5))     

    #One hidden layer: 
    model3.add(Dense(50,  activation = 'tanh',
                #kernel_regularizer=regularizers.l2(0.001),
                #activity_regularizer=regularizers.l1(0.001), 
                kernel_initializer= initializers.glorot_normal(seed=None)))
    #model3.add(BatchNormalization())
    model3.add(Dropout(0.25))

    #Second hidden layer: 
    model3.add(Dense(50, activation = 'tanh', 
                #kernel_regularizer=regularizers.l2(0.01),
                #activity_regularizer=regularizers.l1(0.01), 
                kernel_initializer= initializers.glorot_normal(seed=None)))
    #model3.add(BatchNormalization())
    model3.add(Dropout(0.25))

    #Output layer 
    model3.add(Dense(3, activation='softmax',
                #kernel_regularizer=regularizers.l2(0.01),
                #activity_regularizer=regularizers.l1(0.01), 
                kernel_initializer= initializers.glorot_normal(seed=None)))

    #Setting up to optimize the weights: 
    model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    
    categorical_labels = to_categorical(y_train_all_small[train]-1, num_classes=None)
    model3.fit(small_scaled_bin[train],categorical_labels, epochs=100, batch_size=10, verbose=1)
    
    # evaluate the model
    categorical_labels_test = to_categorical(y_train_all_small[test]-1, num_classes=None)
    scores = model3.evaluate(small_scaled_bin[test], categorical_labels_test, verbose=0)
    print("log loss on test k-th fold:", scores[0])
    cvscores3.append(scores[0])
    
    i+=1

print()
print("mean testing log loss:")
print(numpy.mean(cvscores3), "+/-", numpy.std(cvscores3))

fold number: 1
Epoch 1/100
39469/39469 [==============================] - 8s - loss: 0.7158 - acc: 0.6898     
Epoch 2/100
39469/39469 [==============================] - 7s - loss: 0.6750 - acc: 0.7014     
Epoch 3/100
39469/39469 [==============================] - 7s - loss: 0.6614 - acc: 0.7071     
Epoch 4/100
39469/39469 [==============================] - 7s - loss: 0.6561 - acc: 0.7094     
Epoch 5/100
39469/39469 [==============================] - 7s - loss: 0.6516 - acc: 0.7092     
Epoch 6/100
39469/39469 [==============================] - 8s - loss: 0.6494 - acc: 0.7143     
Epoch 7/100
39469/39469 [==============================] - 7s - loss: 0.6468 - acc: 0.7141     
Epoch 8/100
39469/39469 [==============================] - 7s - loss: 0.6445 - acc: 0.7122     
Epoch 9/100
39469/39469 [==============================] - 7s - loss: 0.6435 - acc: 0.7125     
Epoch 10/100
39469/39469 [==============================] - 7s - loss: 0.6391 - acc: 0.7164     
Epoch 11/100
39469/39469

Interpretation: Scaling the binary predictors doesn't help or hurt the model. 

#### Keras Cross Val Take 4 (all variables scaled, using Batch Normalization):

In [128]:
# define 5-fold cross validation test harness
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
cvscores4 = []
i=1

scale_stand = StandardScaler()

small_scaled_bin = scale_stand.fit_transform(X_train_all_small)

for train, test in kfold.split(small_scaled_bin, y_train_all_small):
    print("fold number:", i)
    model4 = Sequential()
    
    inshape = small_scaled_bin.shape[1]

    #Input layer: 
    model4.add(Dense(50, input_dim= inshape, activation = 'tanh',         
                #kernel_regularizer=regularizers.l2(0.001),
                #activity_regularizer=regularizers.l1(0.001), 
                kernel_initializer= initializers.glorot_normal(seed=None)))
                     
    model4.add(BatchNormalization())
    model4.add(Dropout(0.5))     

    #One hidden layer: 
    model4.add(Dense(50,  activation = 'tanh',
                #kernel_regularizer=regularizers.l2(0.001),
                #activity_regularizer=regularizers.l1(0.001), 
                kernel_initializer= initializers.glorot_normal(seed=None)))
    model4.add(BatchNormalization())
    model4.add(Dropout(0.25))

    #Second hidden layer: 
    model4.add(Dense(50, activation = 'tanh', 
                #kernel_regularizer=regularizers.l2(0.01),
                #activity_regularizer=regularizers.l1(0.01), 
                kernel_initializer= initializers.glorot_normal(seed=None)))
    model4.add(BatchNormalization())
    model4.add(Dropout(0.25))

    #Output layer 
    model4.add(Dense(3, activation='softmax',
                #kernel_regularizer=regularizers.l2(0.01),
                #activity_regularizer=regularizers.l1(0.01), 
                kernel_initializer= initializers.glorot_normal(seed=None)))

    #Setting up to optimize the weights: 
    model4.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    
    
    categorical_labels = to_categorical(y_train_all_small[train]-1, num_classes=None)
    model4.fit(small_scaled_bin[train],categorical_labels, epochs=100, batch_size=10, verbose=1)
    
    # evaluate the model
    categorical_labels_test = to_categorical(y_train_all_small[test]-1, num_classes=None)
    scores = model4.evaluate(small_scaled_bin[test], categorical_labels_test, verbose=0)
    print("log loss on test k-th fold:", scores[0])
    cvscores4.append(scores[0])
    
    i+=1

print()
print("mean testing log loss:")
print(numpy.mean(cvscores4), "+/-", numpy.std(cvscores4))

fold number: 1
Epoch 1/100
39469/39469 [==============================] - 18s - loss: 0.7626 - acc: 0.6768    
Epoch 2/100
39469/39469 [==============================] - 15s - loss: 0.7152 - acc: 0.6927    
Epoch 3/100
39469/39469 [==============================] - 15s - loss: 0.7085 - acc: 0.6941    
Epoch 4/100
39469/39469 [==============================] - 15s - loss: 0.7015 - acc: 0.6952    
Epoch 5/100
39469/39469 [==============================] - 16s - loss: 0.6953 - acc: 0.6948    
Epoch 6/100
39469/39469 [==============================] - 15s - loss: 0.6902 - acc: 0.6974    
Epoch 7/100
39469/39469 [==============================] - 15s - loss: 0.6896 - acc: 0.6960    
Epoch 8/100
39469/39469 [==============================] - 16s - loss: 0.6858 - acc: 0.6995    
Epoch 9/100
39469/39469 [==============================] - 15s - loss: 0.6826 - acc: 0.6997    
Epoch 10/100
39469/39469 [==============================] - 16s - loss: 0.6814 - acc: 0.6986    
Epoch 11/100
39469/39469

KeyboardInterrupt: 

.......Changed the above (which was hte best run) from adam to sgd, results  

#### Keras Cross Val Take 5 (Adding some regularization and early stopping):

In [52]:
# define 5-fold cross validation test 
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
cvscores5 = []
i=1

scale_stand = StandardScaler()

small_scaled_bin = scale_stand.fit_transform(X_train_all_small)

for train, test in kfold.split(small_scaled_bin, y_train_all_small):
    print("fold number:", i)
    model5 = Sequential()
    
    inshape = small_scaled_bin.shape[1]

    #Input layer: 
    model5.add(Dense(50, input_dim= inshape, activation = 'tanh',         
                kernel_regularizer=regularizers.l2(0.001),
                #activity_regularizer=regularizers.l1(0.001), 
                kernel_initializer= initializers.glorot_normal(seed=None)))
                     
    model5.add(BatchNormalization())
    model5.add(Dropout(0.5))     

    #One hidden layer: 
    model5.add(Dense(50,  activation = 'tanh',
                kernel_regularizer=regularizers.l2(0.001),
                #activity_regularizer=regularizers.l1(0.001), 
                kernel_initializer= initializers.glorot_normal(seed=None)))
    model5.add(BatchNormalization())
    model5.add(Dropout(0.5))

    #Second hidden layer: 
    model5.add(Dense(50, activation = 'tanh', 
                #kernel_regularizer=regularizers.l2(0.01),
                #activity_regularizer=regularizers.l1(0.01), 
                kernel_initializer= initializers.glorot_normal(seed=None)))
    model5.add(BatchNormalization())
    model5.add(Dropout(0.5))

    #Output layer 
    model5.add(Dense(3, activation='softmax',
                #kernel_regularizer=regularizers.l2(0.01),
                #activity_regularizer=regularizers.l1(0.01), 
                kernel_initializer= initializers.glorot_normal(seed=None)))

    #Setting up to optimize the weights: 
    model5.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    
    categorical_labels = to_categorical(y_train_all_small[train]-1, num_classes=None)
    model5.fit(small_scaled_bin[train],categorical_labels, epochs=100, batch_size=10, verbose=1)
    
    # evaluate the model
    categorical_labels_test = to_categorical(y_train_all_small[test]-1, num_classes=None)
    scores = model5.evaluate(small_scaled_bin[test], categorical_labels_test, verbose=0)
    print("log loss on test k-th fold:", scores[0])
    cvscores5.append(scores[0])
    
    i+=1

print()
print("mean testing log loss:")
print(numpy.mean(cvscores5), "+/-", numpy.std(cvscores5))

fold number: 1
Epoch 1/100
39469/39469 [==============================] - 18s - loss: 0.8751 - acc: 0.6615    
Epoch 2/100
39469/39469 [==============================] - 16s - loss: 0.7466 - acc: 0.6913    
Epoch 3/100
39469/39469 [==============================] - 17s - loss: 0.7229 - acc: 0.6952    
Epoch 4/100
39469/39469 [==============================] - 17s - loss: 0.7159 - acc: 0.6972    
Epoch 5/100
39469/39469 [==============================] - 17s - loss: 0.7107 - acc: 0.6970    
Epoch 6/100
39469/39469 [==============================] - 16s - loss: 0.7089 - acc: 0.6986    
Epoch 7/100
39469/39469 [==============================] - 17s - loss: 0.7074 - acc: 0.6981    
Epoch 8/100
39469/39469 [==============================] - 16s - loss: 0.7068 - acc: 0.6993    
Epoch 9/100
39469/39469 [==============================] - 16s - loss: 0.7070 - acc: 0.6995    
Epoch 10/100
39469/39469 [==============================] - 17s - loss: 0.7042 - acc: 0.6986    
Epoch 11/100
39469/39469

Interpretation: Too much regularization. 

#### Keras Cross Val Take 6 (no location columns):

In [63]:
# define 5-fold cross validation test harness
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=7)
cvscores6 = []
i=1

scale_stand = StandardScaler()

small_scaled_bin = scale_stand.fit_transform(X_train_all_small[:, :58])

for train, test in kfold.split(small_scaled_bin, y_train_all_small):
    print("fold number:", i)
    model6 = Sequential()
    
    inshape = small_scaled_bin.shape[1]

    #Input layer: 
    model6.add(Dense(50, input_dim= inshape, activation = 'tanh',         
                #kernel_regularizer=regularizers.l2(0.001),
                #activity_regularizer=regularizers.l1(0.001), 
                kernel_initializer= initializers.glorot_normal(seed=None)))
                     
    model6.add(BatchNormalization())
    model6.add(Dropout(0.5))     

    #One hidden layer: 
    model6.add(Dense(50,  activation = 'tanh',
                #kernel_regularizer=regularizers.l2(0.001),
                #activity_regularizer=regularizers.l1(0.001), 
                kernel_initializer= initializers.glorot_normal(seed=None)))
    model6.add(BatchNormalization())
    model6.add(Dropout(0.25))

    #Second hidden layer: 
    model6.add(Dense(50, activation = 'tanh', 
                #kernel_regularizer=regularizers.l2(0.01),
                #activity_regularizer=regularizers.l1(0.01), 
                kernel_initializer= initializers.glorot_normal(seed=None)))
    model6.add(BatchNormalization())
    model6.add(Dropout(0.25))

    #Output layer 
    model6.add(Dense(3, activation='softmax',
                #kernel_regularizer=regularizers.l2(0.01),
                #activity_regularizer=regularizers.l1(0.01), 
                kernel_initializer= initializers.glorot_normal(seed=None)))

    #Setting up to optimize the weights: 
    model6.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    
    categorical_labels = to_categorical(y_train_all_small[train]-1, num_classes=None)
    model6.fit(small_scaled_bin[train],categorical_labels, epochs=100, batch_size=10, verbose=1)
    
    # evaluate the model
    categorical_labels_test = to_categorical(y_train_all_small[test]-1, num_classes=None)
    scores = model6.evaluate(small_scaled_bin[test], categorical_labels_test, verbose=0)
    print("log loss on test k-th fold:", scores[0])
    cvscores6.append(scores[0])
    
    i+=1

print()
print("mean testing log loss:")
print(numpy.mean(cvscores6), "+/-", numpy.std(cvscores6))

fold number: 1
Epoch 1/100
32891/32891 [==============================] - 13s - loss: 0.7672 - acc: 0.6687    
Epoch 2/100
32891/32891 [==============================] - 9s - loss: 0.7013 - acc: 0.6944     
Epoch 3/100
32891/32891 [==============================] - 9s - loss: 0.6906 - acc: 0.6985     
Epoch 4/100
32891/32891 [==============================] - 9s - loss: 0.6847 - acc: 0.6995     
Epoch 5/100
32891/32891 [==============================] - 9s - loss: 0.6795 - acc: 0.7017     
Epoch 6/100
32891/32891 [==============================] - 8s - loss: 0.6771 - acc: 0.7044     
Epoch 7/100
32891/32891 [==============================] - 8s - loss: 0.6763 - acc: 0.7014     
Epoch 8/100
32891/32891 [==============================] - 8s - loss: 0.6711 - acc: 0.7049     
Epoch 9/100
32891/32891 [==============================] - 8s - loss: 0.6727 - acc: 0.7054     
Epoch 10/100
32891/32891 [==============================] - 8s - loss: 0.6694 - acc: 0.7053     
Epoch 11/100
32891/32891

KeyboardInterrupt: 

Interpretation: The locations column adds a lot of information. Log losses are .02 points higher without it. 

#### Keras Cross Val Take 7 (full dataset):

In [126]:
# define 5-fold cross validation test harness
from sklearn.model_selection import StratifiedKFold



kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=7)
cvscores7 = []
i=1

scale_stand = StandardScaler()

scaled_bin = scale_stand.fit_transform(X_train_all)

for train, test in kfold.split(scaled_bin, y_train_all):
    print("fold number:", i)
    model7 = Sequential()
    
    #stop early if testing error gets large: 
    callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')

    
    inshape = scaled_bin.shape[1]

    #Input layer: 
    model7.add(Dense(20, input_dim= inshape, activation = 'tanh',         
                kernel_regularizer=regularizers.l2(0.00001),
                activity_regularizer=regularizers.l1(0.00001), 
                kernel_initializer= initializers.glorot_normal(seed=None)))
                     
    model7.add(BatchNormalization())
    model7.add(Dropout(0.5))     

    #One hidden layer: 
    model7.add(Dense(20,  activation = 'tanh',
                kernel_regularizer=regularizers.l2(0.00001),
                activity_regularizer=regularizers.l1(0.00001), 
                kernel_initializer= initializers.glorot_normal(seed=None)))
    model7.add(BatchNormalization())
    model7.add(Dropout(0.5))

    #Second hidden layer: 
    model7.add(Dense(20, activation = 'tanh', 
                #kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.0001), 
                kernel_initializer= initializers.glorot_normal(seed=None)))
    model7.add(BatchNormalization())
    model7.add(Dropout(0.5))
    
    #Third hidden layer: 
    model7.add(Dense(20, activation = 'tanh', 
                kernel_regularizer=regularizers.l2(0.00001),
                activity_regularizer=regularizers.l1(0.00001), 
                kernel_initializer= initializers.glorot_normal(seed=None)))
    model7.add(BatchNormalization())
    model7.add(Dropout(0.5))

    #Output layer 
    model7.add(Dense(3, activation='softmax',
                kernel_regularizer=regularizers.l2(0.0001),
                activity_regularizer=regularizers.l1(0.00001), 
                kernel_initializer= initializers.glorot_normal(seed=None)))

    #Setting up to optimize the weights: 
    model7.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    
    categorical_labels = to_categorical(y_train_all[train]-1, num_classes=None)
    model7.fit(scaled_bin[train],categorical_labels, epochs=5, batch_size=10, verbose=1)
    
    # evaluate the model
    categorical_labels_test = to_categorical(y_train_all[test]-1, num_classes=None)
    scores = model7.evaluate(scaled_bin[test], categorical_labels_test, verbose=0)
    
    print("log loss on test k-th fold:", scores[0])
    cvscores7.append(scores[0])
    
    i+=1

print()
print("mean testing log loss:")
print(numpy.mean(cvscores7), "+/-", numpy.std(cvscores7))

fold number: 1
Epoch 1/5
32891/32891 [==============================] - 26s - loss: 0.8484 - acc: 0.6589    
Epoch 2/5
32891/32891 [==============================] - 22s - loss: 0.7627 - acc: 0.6945    
Epoch 3/5
32891/32891 [==============================] - 23s - loss: 0.7551 - acc: 0.6944    
Epoch 4/5
32891/32891 [==============================] - 23s - loss: 0.7519 - acc: 0.6937    
Epoch 5/5
32891/32891 [==============================] - 22s - loss: 0.7449 - acc: 0.6938    
log loss on test k-th fold: 0.747604794497
fold number: 2
Epoch 1/5
32891/32891 [==============================] - 26s - loss: 0.8606 - acc: 0.6548    
Epoch 2/5
32891/32891 [==============================] - 22s - loss: 0.7637 - acc: 0.6935    
Epoch 3/5
32891/32891 [==============================] - 23s - loss: 0.7560 - acc: 0.6941    
Epoch 4/5
14510/32891 [============>.................] - ETA: 12s - loss: 0.7429 - acc: 0.6977

KeyboardInterrupt: 

In [119]:
categorical_labels.shape

(32891, 3)

In [120]:
y_train_all.shape

(49337,)

In [122]:
scaled_bin[test].shape

(16446, 837)

In [123]:
small_scaled_bin[test].shape

(16446, 58)

#### Keras grid search: 

In [363]:
#Using Grid Search to tune the parameters: 

def create_model(optimizer='rmsprop', init='glorot_uniform'):
    model = Sequential()
    model.add(Dense(12, input_dim= X_train.shape[1], activation='tanh'))
    model.add(Dropout(0.25))
    model.add(Dense(X_train.shape[1], activation='tanh'))
    model.add(Dropout(0.25))
    model.add(Dense(3, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', metrics = ['accuracy'], optimizer= 'sgd')
    return model
 
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# grid search epochs, batch size and optimizer
optimizers = ['rmsprop', 'adam']
init = ['glorot_uniform', 'normal', 'uniform']
batches = [5, 10, 20]

param_grid = dict(optimizer=optimizers, batch_size=batches, init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid)

In [364]:
#grid_result = grid.fit(X_train, categorical_labels )

KeyboardInterrupt: 

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))

## Areas of extension:

- Dynamic sampling to balance the classes